In [2]:
import ee

ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AfJohXlETyvRvc8g_CU8xYVJ8naQ1Wd2SfWTWWgX7ZEv-a6dttVDchQG31M



Successfully saved authorization token.


In [31]:
import folium


In [11]:
# import ee
# ee.Initialize()

# Load the ImageCollection
gsw_monthly = ee.ImageCollection('JRC/GSW1_4/MonthlyHistory')
# print('gsw monthly:', gsw_monthly)

# Function to calculate monthly max values within each year
yearlist = ee.List.sequence(2000, 2021)
reducer = ee.Reducer.max()

# Function to create annual composites
def create_yearly_composite(year):
    year = ee.Number(year)
    yearCol = gsw_monthly.filter(ee.Filter.calendarRange(year, year, 'year'))
    yearMax = yearCol.reduce(reducer)

    imgList = yearCol.aggregate_array('system:index')
    n_img = imgList.size()
    nBands = yearMax.bandNames().size()

    return yearMax.set({
        'year': year,
        'image_list': imgList,
        'n_bands': nBands,
        'n_img': n_img,
        'method': 'max value',
        'system:time_start': year
    })

yearCompList = yearlist.map(create_yearly_composite)

# Convert the annual composite image list to an ImageCollection
annual_max_water = ee.ImageCollection.fromImages(yearCompList).filter(ee.Filter.gt('n_bands', 0))
# print('annual_max_water', annual_max_water)

# Convert to list
annual_list = annual_max_water.toList(annual_max_water.size())
# print('annual_list', annual_list)

# Function to calculate the difference between two images
def difference(img1, img2):
    def mask_only_water(img):
        return img.updateMask(img.eq(2)).multiply(0).add(1).unmask(0)

    img1_mask = mask_only_water(img1)
    img2_mask = mask_only_water(img2)

    diff = img2_mask.subtract(img1_mask).copyProperties(img1_mask, ['max value', 'n_img', 'system:time_start', 'year'])
    img_diff = ee.Image(diff)
    return img_diff

year1L = 2012
year2L = 2013

imgtest1 = ee.Image(annual_list.get(year1L - 2012))
# print(imgtest1)
imgtest2 = ee.Image(annual_list.get(year2L - 2013))

diff = difference(imgtest1, imgtest2)
# print(diff)

# Additional code for visualization and analysis would go here, if needed.


In [36]:
# Load datasets
era5_land_temp = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").select('temperature_2m')
era5_land_precip = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").select('total_precipitation')
modis_snow_cover = ee.ImageCollection("MODIS/006/MOD10A1").select('NDSI_Snow_Cover')
# gsw_monthly = ee.ImageCollection('JRC/GSW1_4/MonthlyHistory')

# # Function to upscale and aggregate datasets
# def upscale_and_aggregate(dataset, target_scale=9000, reducer=ee.Reducer.sum()):
#     return dataset.map(lambda image: image.reduceResolution(reducer, False).reproject({
#         'crs': image.projection().crs(),
#         'scale': target_scale
#     }))

# # Upscale GSW and MODIS datasets
# upscaled_gsw = upscale_and_aggregate(ee.ImageCollection([diff,]))
# upscaled_snow = upscale_and_aggregate(modis_snow_cover)

def upscale_and_aggregate(dataset, target_scale=9000, reducer=ee.Reducer.mean()):
    def resample(image):
        # Use the native projection of the image
        proj = image.projection()
        return image.reduceResolution(
            reducer=reducer, 
            maxPixels=65535
        ).reproject(
            crs=proj, 
            scale=target_scale
        )
    return dataset.map(resample)

# Apply the function to GSW and MODIS datasets
upscaled_gsw = upscale_and_aggregate(gsw_monthly)
upscaled_snow = upscale_and_aggregate(modis_snow_cover)


In [37]:
# Define the time range
start_year = 2012
end_year = 2013
yearlist = ee.List.sequence(start_year, end_year)

# Function to create annual composites
def create_annual_composite(collection, year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = start_date.advance(1, 'year')
    return collection.filterDate(start_date, end_date).mean().set('year', year)

# Create annual composites for each dataset
annual_temp = ee.ImageCollection(yearlist.map(lambda year: create_annual_composite(era5_land_temp, year)))
annual_precip = ee.ImageCollection(yearlist.map(lambda year: create_annual_composite(era5_land_precip, year)))
annual_gsw = ee.ImageCollection(yearlist.map(lambda year: create_annual_composite(upscaled_gsw, year)))
annual_snow = ee.ImageCollection(yearlist.map(lambda year: create_annual_composite(upscaled_snow, year)))


In [38]:
def combine_annual_data(year):
    temp_image = annual_temp.filter(ee.Filter.eq('year', year)).first()
    precip_image = annual_precip.filter(ee.Filter.eq('year', year)).first()
    gsw_image = annual_gsw.filter(ee.Filter.eq('year', year)).first()
    snow_image = annual_snow.filter(ee.Filter.eq('year', year)).first()

    # Combine into a single image
    combined_image = temp_image.addBands([precip_image, gsw_image, snow_image]).set('year', year)
    return combined_image

combined_annual_data = ee.ImageCollection(yearlist.map(combine_annual_data))


In [47]:
# Define an example ROI. Replace this with your actual ROI.
roi = ee.Geometry.Rectangle([70, 45, 110, 23])


# Function to clip each image to the ROI
def clip_to_roi(image):
    return image.clip(roi)

# Clip all images in the collection
combined_annual_data = combined_annual_data.map(clip_to_roi)


In [63]:
# Define the corners of the large ROI
top_left = (70, 45)
bottom_right = (110, 23)

# Define the grid size
cols = 5  # Number of columns
rows = 4  # Number of rows

# Calculate the step size for each grid cell
width_step = (bottom_right[0] - top_left[0]) / cols
height_step = (top_left[1] - bottom_right[1]) / rows


In [64]:
def create_grid(top_left, bottom_right, width_step, height_step, cols, rows):
    grid = []
    for i in range(cols):
        for j in range(rows):
            # Calculate corners of the cell
            left = top_left[0] + i * width_step
            right = left + width_step
            top = top_left[1] - j * height_step
            bottom = top - height_step

            # Create a rectangle for this cell
            cell = ee.Geometry.Rectangle([left, bottom, right, top])
            grid.append(cell)
    return grid

grid = create_grid(top_left, bottom_right, width_step, height_step, cols, rows)


In [67]:
def export_data(img, name, section):
    task = ee.batch.Export.image.toDrive(**{
        'image':img
        ,'description':name+f'_{section}'
        ,'folder':'EarthEngineData'
        ,'region':section
        ,'fileFormat': 'GeoTIFF'
        ,'scale': 9000
        ,'maxPixels': 1e13
    })
    task.start()
    print(f'task {section} sent to Google.')

In [66]:
for i, section in enumerate(grid):
    task = ee.batch.Export.image.toDrive(**{
        'image': combined_annual_data,  # Replace with your 'difference' image
        # 'description': f'High_Mountain_Asia_Difference_Section_{i}',
        'description': f'Difference_2012_Section_{i}',
        'folder': 'EarthEngineData',
        'scale': 30,  # Adjust as necessary
        'region': section,
        'fileFormat': 'GeoTIFF',
        'maxPixels': 1e13
    })
    task.start()


In [48]:
# # Flatten the ImageCollection into a single Image with multiple bands
# def flatten_to_single_image(collection):
#     first_image = ee.Image(collection.first()).select([])
#     return ee.Image(collection.iterate(lambda img, acc: ee.Image(acc).addBands(img), first_image))

# combined_image = flatten_to_single_image(combined_annual_data)


In [49]:
# # Select the combined image for a specific year
# selected_year = 2012  # Example year
# selected_image = combined_annual_data.filter(ee.Filter.eq('year', selected_year)).first()


In [50]:
# selected_image.bandNames().getInfo()

['temperature_2m', 'total_precipitation', 'water', 'NDSI_Snow_Cover']

In [60]:
# # Define visualization parameters
# vis_params_temp = {
#     'min': 270, 'max': 310, 'palette': ['blue', 'green', 'yellow', 'red']  # Example parameters for temperature
# }
# vis_params_precip = {
#     'min': 0, 'max': 100, 'palette': ['white', 'blue']  # Example parameters for precipitation
# }
# vis_params_gsw = {
#     # 'min': 0, 'max': 1, 'palette': ['white', 'red']  # Example parameters for GSW
#     'palette': ['white', 'red']  # Example parameters for GSW
# }
# vis_params_snow = {
#     'min': 0, 'max': 100, 'palette': ['white', 'blue']  # Example parameters for snow cover
# }


In [61]:
# # Create a Folium map
# map_center = [45, 70]  # Adjust as necessary
# m = folium.Map(location=map_center, zoom_start=4)

# # Function to add a layer to the map
# def add_ee_layer(self, ee_image_object, vis_params, name):
#     map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
#     folium.TileLayer(
#         tiles=map_id_dict['tile_fetcher'].url_format,
#         attr='Map data © Google Earth Engine',
#         name=name,
#         overlay=True,
#         control=True
#     ).add_to(self)

# # Add custom method to folium Map
# folium.Map.add_ee_layer = add_ee_layer

# # Add layers
# # # m.add_ee_layer(selected_image.select('temperature_2m'), vis_params_temp, 'Temperature')
# # m.add_ee_layer(selected_image.select('total_precipitation'), vis_params_precip, 'Precipitation')
# m.add_ee_layer(selected_image.select('water'), vis_params_gsw, 'GSW')  # Replace 'GSW_band_name' with the actual band name
# # m.add_ee_layer(selected_image.select('NDSI_Snow_Cover'), vis_params_snow, 'Snow Cover')  # Replace 'Snow_band_name' with the actual band name

# # Add a layer control panel to the map
# folium.LayerControl().add_to(m)

# # Display the map
# m


In [62]:
# !pip install ipyleaflet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 4.6 MB/s eta 0:00:0000:0100:01


In [18]:
# # Define your region of interest (ROI)
# roi = ee.Geometry.Rectangle([70, 45, 110, 23])  # Adjust as necessary

# # Create random points within the ROI
# points = ee.FeatureCollection.randomPoints(region=roi, points=1000)  # Adjust the number of points as necessary


In [19]:
# # Sample the combined image at the random points
# sampled_points = combined_image.sampleRegions(**{
#     'collection': points,
#     'properties': ['system:index'],  # Include other properties if available
#     'scale': 9000
# })


In [24]:
# print(combined_image.bandNames().getInfo())

In [27]:
# # Define a pseudo-label for demonstration purposes
# pseudo_label = 'water balance'  # Example: use temperature as a pseudo-label

# # Extract band names excluding the pseudo-label
# features = combined_image.bandNames().remove(pseudo_label)

# # Prepare the training data
# training_data = sampled_points.select(features.add(pseudo_label))


In [28]:
# # Example: training_data = ee.FeatureCollection('path/to/your/training_data')
# label = 'water_balance'  # Replace with your actual label property
# features = combined_image.bandNames()  # Get all the band names from the combined image

# # Sample the image for training the model
# training = combined_image.sampleRegions(**{
#     'collection': training_data,
#     'properties': [label],
#     'scale': 9000
# })
